In [85]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [4]:
from pickle import load

stories = load(open('../data/review_dataset.pkl', 'rb'))
print('Loaded Stories %d' % len(stories))
print(type(stories))

Loaded Stories 568411
<class 'list'>


In [10]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
#n =0
#m = 0

for story in stories:
    input_text = story['story']
#    if n<3:
#        print('first round input_text', input_text)
    target_text = ''
    for highlight in story['highlights']:
        target_text += highlight
#        if n<5:
#            print('target_text inside for loop',target_text)
#            n += 1

    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
#    if m<3:
#        print('target_text after tn',target_text)
    input_texts.append(input_text)
    target_texts.append(target_text)
#    if m<3:
#        print('target_text after append',target_text)  
#        print('input_text after append',input_text)
#        m +=1
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [62]:
###
input_texts = input_texts[0:10000]

In [63]:
len(input_texts)

10000

In [64]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 84
Number of unique output tokens: 58
Max sequence length for inputs: 5437
Max sequence length for outputs: 132


In [69]:
num_sample = len(input_texts)

In [70]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [71]:
encoder_input_data = np.zeros(
    (num_sample, max_encoder_seq_length, num_encoder_tokens), 
    dtype ='float32')

In [72]:
decoder_input_data = np.zeros(
    (num_sample, max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (num_sample, max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [73]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [2]:
###Try new
import numpy as np
import csv
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model

In [3]:
# load dataset
with open('../data/food_review.csv') as f:
    reader = csv.reader(f)
    l = list(reader)

In [4]:
en = list()
de = list()
for i in range(1, len(l)):
    en.append(l[i][0])
    de.append(l[i][1])

In [5]:
en = en[0:10000]
de = de[0:10000]

In [21]:
# Max sequence length for inputs: 5437
# Max sequence length for outputs: 132
max_encoder_seq_length = 5437
max_decoder_seq_length = 132

In [51]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

In [23]:
# ##
# # tokenizer = Tokenizer() #num_words= vocabulary_size
# # tokenizer.fit_on_texts(all_glove_words) #data_df['phrase']

# en_tokenize = Tokenizer(num_words=max_encoder_seq_length, char_level=True)
# en_tokenize.fit_on_texts(en) 
# en_token = en_tokenize.texts_to_matrix(en)

# de_tokenize = Tokenizer(num_words=max_decoder_seq_length, char_level=True)
# de_tokenize.fit_on_texts(de) 
# de_token = de_tokenize.texts_to_matrix(de)

In [36]:
de_token.shape

(10000, 132)

In [35]:
de_token[0]

array([0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# # Use sklearn utility to convert label strings to numbered index
# encoder = LabelEncoder()
# encoder.fit(allwords)
# y_train = encoder.transform(train_label)
# y_test = encoder.transform(test_label)

# # Converts the labels to a one-hot representation
# num_classes = np.max(y_train) + 1
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)

In [49]:
### New 

tokenizer = Tokenizer(num_words=max_encoder_seq_length, char_level=True)
tokenizer.fit_on_texts(en+de)

In [79]:
# Max sequence length for inputs: 5437
# Max sequence length for outputs: 132
max_encoder_seq_length = 5437
max_decoder_seq_length = 132

In [80]:
source_token = tokenizer.texts_to_sequences(en)
target_token = tokenizer.texts_to_sequences(de)
# target_in_token = [0] + target_token[:-1]

source_padded = pad_sequences(source_token, maxlen=max_encoder_seq_length, padding = "post")
target_padded = pad_sequences(target_token, maxlen=max_decoder_seq_length, padding = "post")


In [69]:
print(len(target_in_token[2]))
print((target_in_token[2]))
# print(len(source_padded[0]))
# print(source_padded[0])

17
[9, 5, 4, 1, 3, 7, 1, 3, 11, 23, 2, 6, 4, 8, 7, 2, 11]


In [ ]:
# encoder_input_data
# decoder_input_data
# decoder_target_data

In [1]:
###Final!!!!!
# # generate a sequence of random integers
# def generate_sequence(length, n_unique):
#     return [randint(1, n_unique-1) for _ in range(length)]

# cardinality = 89 + 1
# n_in = 6
# n_out = 3
n_samples =2

# Number of unique input tokens: 84
# Number of unique output tokens: 58
num_encoder_tokens = 84
num_decoder_tokens = 58

# prepare data for the LSTM

X1, X2, y = list(), list(), list()
for i in range(n_samples):
# generate source sequence
    source = source_padded[i] #generate_sequence(n_in, cardinality)
#     print('source',source)
    # define target sequence
    target = target_padded[i] #source[:n_out]
#     target.reverse()
#     print('target',target)
    # create padded input target sequence
    target_in = [0] + target[:-1]
#     print('target_in', target_in)
    # encode
    src_encoded = to_categorical(source, num_classes=num_encoder_tokens)
    tar_encoded = to_categorical(target, num_classes=num_decoder_tokens)
    tar2_encoded = to_categorical(target_in, num_classes=num_decoder_tokens)
#     print(src_encoded.shape)
#     print('src_encoded',src_encoded)
    # store
    X1.append(src_encoded)
    X2.append(tar2_encoded)
    y.append(tar_encoded)
#     print(array(X1))
    
# X1 = array(X1)
# X2 = array(X2)
# y = array(y)
encoder_input_data = array(X1)
decoder_input_data = array(X2)
decoder_target_data = array(y)

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

# # decode a one hot encoded string
# def one_hot_decode(encoded_seq):
#     return [argmax(vector) for vector in encoded_seq]
 

# print(X1.shape, X2.shape, y.shape)
# print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

NameError: name 'source_padded' is not defined

In [95]:
# print(X1[1].shape)

(1, 5437, 84)


In [ ]:
import numpy as np

In [13]:
from random import randint
from numpy import array
from tensorflow.keras.utils import to_categorical

# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique-1) for _ in range(length)]

cardinality = 50 + 1
n_in = 6
n_out = 3
n_samples =1


# prepare data for the LSTM

X1, X2, y = list(), list(), list()
def get_dataset(n_in, n_out, cardinality, n_samples):
    for _ in range(n_samples):
    # generate source sequence
        source = generate_sequence(n_in, cardinality) #generate_sequence(n_in, cardinality)
#         print('source',source)
        # define target sequence
        target = source[:n_out] #source[:n_out]
        target.reverse()
        print(len(target))
        print(type(target))
        print('target',target)
        # create padded input target sequence
        target_in = [0] + target[:-1]
        print(len(target_in))
        print(type(target_in))
        print('target_in', target_in)
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
#         print(src_encoded.shape)
#         print('src_encoded',src_encoded)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)
X1, X2, y = get_dataset(n_in, n_out, cardinality, n_samples)

# print(X1.shape, '\n', X2.shape, '\n',y.shape)
# X1 = np.asarray(X1)
# X2 = np.asarray(X2)
# y = np.asarray(y)

# print(array(X1).shape)

# # # decode a one hot encoded string
# # def one_hot_decode(encoded_seq):
# #     return [argmax(vector) for vector in encoded_seq]
 

print(X1.shape, X2.shape, y.shape)
# # print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

3
<class 'list'>
target [5, 9, 28]
3
<class 'list'>
target_in [0, 5, 9]
(1, 6, 51) (1, 3, 51) (1, 3, 51)


In [ ]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
 
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique-1) for _ in range(length)]
 
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define padded target sequence
        target = source[:n_out]
        target.reverse()
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)
 
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model
 
# # generate target given source sequence
# def predict_sequence(infenc, infdec, source, n_steps, cardinality):
# 	# encode
# 	state = infenc.predict(source)
# 	# start of sequence input
# 	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
# 	# collect predictions
# 	output = list()
# 	for t in range(n_steps):
# 		# predict next char
# 		yhat, h, c = infdec.predict([target_seq] + state)
# 		# store prediction
# 		output.append(yhat[0,0,:])
# 		# update state
# 		state = [h, c]
# 		# update target sequence
# 		target_seq = yhat
# 	return array(output)
 
# # decode a one hot encoded string
# def one_hot_decode(encoded_seq):
# 	return [argmax(vector) for vector in encoded_seq]
 
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(X1.shape,X2.shape,y.shape)
# train model
train.fit([X1, X2], y, epochs=1)

(100000, 6, 51) (100000, 3, 51) (100000, 3, 51)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [110]:
# from random import randint
# from numpy import array
# from numpy import argmax
# from tensorflow.keras.utils import to_categorical
 
# # generate a sequence of random integers
# def generate_sequence(length, n_unique):
#     a = [randint(1, n_unique-1) for _ in range(length)]
#     print('a', type(a))
#     return a
 
# # prepare data for the LSTM
# def get_dataset(n_in, n_out, cardinality, n_samples):
#     X1, X2, y = list(), list(), list()
#     for _ in range(n_samples):
#     # generate source sequence
#         source = generate_sequence(n_in, cardinality)
#         # define target sequence
#         target = source[:n_out]
#         target.reverse()
#         # create padded input target sequence
#         target_in = [0] + target[:-1]
#         # encode
#         print(type(source))
#         print(source)
#         src_encoded = to_categorical(source, num_classes=cardinality)
#         tar_encoded = to_categorical([target], num_classes=cardinality)
#         tar2_encoded = to_categorical([target_in], num_classes=cardinality)
#         # store
#         print(src_encoded.shape)
#         X1.append(src_encoded)
#         X2.append(tar2_encoded)
#         y.append(tar_encoded)
#     return array(X1), array(X2), array(y)
 
# # decode a one hot encoded string
# def one_hot_decode(encoded_seq):
#     return [argmax(vector) for vector in encoded_seq]
 
# # configure problem
# n_features = 50 + 1
# n_steps_in = 6
# n_steps_out = 3
# # generate a single source and target sequence
# X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
# print(X1.shape, X2.shape, y.shape)
# print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

a <class 'list'>
<class 'list'>
[8, 20, 27, 19, 41, 39]
(6, 51)
(1, 6, 51) (1, 1, 3, 51) (1, 1, 3, 51)
X1=[8, 20, 27, 19, 41, 39], X2=[0], y=[27]


In [115]:
batch_size = 64  # Batch size for training.
epochs = 30  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.

In [116]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('model2.h5')


Train on 8000 samples, validate on 2000 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30


In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [ ]:
def define_models(n_input, n_output, n_units):
# define training encoder
encoder_inputs = Input(shape=(None, n_input))
encoder = LSTM(n_units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
# define training decoder
decoder_inputs = Input(shape=(None, n_output))
decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(n_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# define inference encoder
encoder_model = Model(encoder_inputs, encoder_states)
# define inference decoder
decoder_state_input_h = Input(shape=(n_units,))
decoder_state_input_c = Input(shape=(n_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,  initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
# return all models
return model, encoder_model, decoder_model

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
batch_size=batch_size,
epochs=epochs,
validation_split=0.2)
# Save model
model.save('/deeplearning-keras/ch09/summarization/model2.h5')

In [ ]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
# encode
state = infenc.predict(source)
# start of sequence input
target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
# collect predictions
output = list()
for t in range(n_steps):
# predict next char
yhat, h, c = infdec.predict([target_seq] + state)
# store prediction
output.append(yhat[0,0,:])
# update state
state = [h, c]
# update target sequence
target_seq = yhat
return array(output)

In [ ]:
# def decode_sequence(input_seq):
#     # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1.
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     # Populate the first character of target sequence with the start character.
#     target_seq[0, 0, target_token_index['\t']] = 1.

#     # Sampling loop for a batch of sequences
#     # (to simplify, here we assume a batch of size 1).
#     stop_condition = False
#     decoded_sentence = ''
#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict(
#             [target_seq] + states_value)

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_char = reverse_target_char_index[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length
#         # or find stop character.
#         if (sampled_char == '\n' or
#            len(decoded_sentence) > max_decoder_seq_length):
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.

#         # Update states
#         states_value = [h, c]

#     return decoded_sentence


# for seq_index in range(100):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decoded_sentence)
